In [59]:
import islpy as isl
from src.io_tools import *
from src.graph_tools import *
from src.swap_tools import *

In [60]:
json_file_path = 'benchmarks/polyhedral/cases/bigd.json'

data = json_file_to_isl(json_file_path)

In [61]:
qops = data["Qops"]

domain, access, schedule = filter_multi_qubit_gates(data["domain"], data["read_dependencies"], data["schedule"])


In [62]:
physical_qubits_domain = isl.Set(f"{{ [i] : 1 <= i <= {qops} }}")

In [63]:
    
graph = [
    [1, 5],           # Q[1] is connected to Q[2] and Q[6]
    [0, 2, 6],        # Q[2] is connected to Q[1], Q[3], and Q[7]
    [1, 3, 7],        # Q[3] is connected to Q[2], Q[4], and Q[8]
    [2, 4, 8],        # Q[4] is connected to Q[3], Q[5], and Q[9]
    [3, 9],           # Q[5] is connected to Q[4] and Q[10]
    [0, 6, 10],       # Q[6] is connected to Q[1], Q[7], and Q[11]
    [1, 5, 7, 11],    # Q[7] is connected to Q[2], Q[6], Q[8], and Q[12]
    [2, 6, 8, 12],    # Q[8] is connected to Q[3], Q[7], Q[9], and Q[13]
    [3, 7, 9, 13],    # Q[9] is connected to Q[4], Q[8], Q[10], and Q[14]
    [4, 8, 14],       # Q[10] is connected to Q[5], Q[9], and Q[15]
    [5, 11, 15],      # Q[11] is connected to Q[6], Q[12], and Q[16]
    [6, 10, 12, 16],  # Q[12] is connected to Q[7], Q[11], Q[13], and Q[17]
    [7, 11, 13, 17],  # Q[13] is connected to Q[8], Q[12], Q[14], and Q[18]
    [8, 12, 14, 18],  # Q[14] is connected to Q[9], Q[13], Q[15], and Q[19]
    [9, 13, 19],      # Q[15] is connected to Q[10], Q[14], and Q[20]
    [10, 16],         # Q[16] is connected to Q[11] and Q[17]
    [11, 15, 17],     # Q[17] is connected to Q[12], Q[16], and Q[18]
    [12, 16, 18],     # Q[18] is connected to Q[13], Q[17], and Q[19]
    [13, 17, 19],     # Q[19] is connected to Q[14], Q[18], and Q[20]
    [14, 18]          # Q[20] is connected to Q[15] and Q[19]
]

backend_disconnected_edges =  extract_edges_map(graph)
shortest_paths = extract_shortest_paths(graph)

In [64]:
initial_mapping = isl.Map(f"{{ q[i] -> [i] : 1<=i<={qops} }}")

In [65]:
mapping = initial_mapping
swap_count = 0
programme = schedule

In [66]:
import time

In [67]:
iteration = 0
start = time.time()
while not programme.is_empty():
    print(f"Iteration {iteration}")
    print(programme.domain())
    iteration += 1
    programme_mapped = programme.apply_range(mapping)
    programme_access = programme_mapped.flat_range_product(programme_mapped).intersect_range(isl.Set("{ [i, j] : i > j }"))
    first_disconnected_edge = programme_access.intersect_range(backend_disconnected_edges).domain().lexmin()
    if first_disconnected_edge.is_empty():
        break

    disconected_qubits = programme_mapped.intersect_domain(first_disconnected_edge).range().as_set()
    q1 , q2 = eval(disconected_qubits.to_str().replace('[','').replace(']','').replace(';',','))
    swap_count += shortest_paths[q1]['costs'][q2] -1
    new_domain = programme.domain().as_set().lex_gt_set(first_disconnected_edge.as_set()).domain()
    programme = programme.intersect_domain(new_domain).coalesce()
    swap_map = swaps_to_isl_map(shortest_paths[q1]['paths'][q2])
    mapping = apply_swaps_to_logical_qubits_map(swap_map,mapping,physical_qubits_domain)
print(f"Time taken: {time.time() - start}")


Iteration 0
{ [i0] : 5 <= i0 <= 8 and (i0 >= 7 or i0 >= 7 or i0 <= 6); [9] }
Iteration 1
{ [i0] : 7 <= i0 <= 8; [6]; [9] }
Iteration 2
{ [i0] : 7 <= i0 <= 8; [9] }
Iteration 3
{ [9]; [8] }
Iteration 4
{ [9] }
Time taken: 0.024967432022094727


In [47]:
print(programme.domain())


{ S8[i0] : 0 <= i0 <= 1; S6[i0, i1] : i0 >= 0 and 0 <= i1 <= 1 - i0 }


In [19]:
isl.UnionMap("{ q[1] -> Q[2] ; q[1] -> Q[3] }").as_map()

Map("{ q[1] -> Q[3]; q[1] -> Q[2] }")

In [21]:
map.flat_range_product(map)

eval("{ 1,2}")

{1, 2}